In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re

In [122]:
# reading the datasets from the directory Datasets/SocioEconomic
# read the pop_stab dataframe up to 214 rows only
pop_stab =  pd.read_csv('../Datasets/SocioEconomic/political_stability_1996_2021.csv', nrows=214)
china_hdi_le =  pd.read_csv('../Datasets/SocioEconomic/China_HDI_LE1990_2021.csv')
s_korea_hdi_le = pd.read_csv('../Datasets/SocioEconomic/SKorea_HDI_LE_1990_2021.csv')
us_hdi_le =  pd.read_csv('../Datasets/SocioEconomic/US_HDI_LE_1990_2021.csv')
pop =  pd.read_csv('../Datasets/SocioEconomic/population_1960_2022.csv')
urban_pop =  pd.read_csv('../Datasets/SocioEconomic/Urban_pop_WorldBank_1960_2021.csv')
gdp =  pd.read_csv('../Datasets/SocioEconomic/GDP_1960_2020.csv')
oil_prices = pd.read_csv('../Datasets/SocioEconomic/crude_oil_price_1983_2023.csv')
inflation =  pd.read_csv('../Datasets/SocioEconomic/Inflation_data_1970_2022.csv', encoding='ISO-8859-1')

## Cleaning Datasets
### Political Stability

The dataset under consideration contains information about the "Political Stability and Absence of Violence/Terrorism" index for various countries from the year 1996 to 2021. The index, coded as 'PV.EST', is measured on a scale from -2.5 to 2.5, with a higher value indicating more stable political conditions and less violence or terrorism. A lower score indicates a less stable political environment and higher levels of violence or terrorism.

The dataset includes 217 countries. However, for the purpose of our analysis, we will focus on three specific countries: USA, China (CHN), and South Korea (KOR).

Our data cleaning and preparation steps include:

- **Country selection**: We're focusing our analysis on three specific countries (USA, CHN, KOR). We filter the data to include only these countries.

- **Renaming columns**: The year columns are initially named in the format 'YYYY [YRYYYY]'. We rename these to just 'YYYY' for simplicity and ease of access.

- **Data type conversion**: We ensure that the values in the year columns are of numeric type (float) for proper analysis.

- **Handling missing years**: Our dataset does not include data for the years 1997, 1999, and 2001. We handle this by computing the missing values as the average of the previous and following years. This is done under the assumption that the political stability index changes smoothly over time.

- **Sorting columns**: After inserting the missing years, we sort our columns to maintain a chronological order of years.



In [123]:
# defining a list of countries
countries_of_interest = ['USA', 'CHN', 'KOR']
# get all the rows for each country of interest
pop_stab = pop_stab[pop_stab['Country Code'].isin(countries_of_interest)]
# renaming the columns by removing [YR]
pop_stab = pop_stab.rename(columns={col: re.sub(r'\s*\[YR.*\]', '', col) for col in pop_stab.columns[4:]})
# Change the type of year columns to float
for col in pop_stab.columns[4:]:
    pop_stab[col] = pop_stab[col].astype(float)

In [124]:
pop_stab.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code', '1996',
       '1998', '2000', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021'],
      dtype='object')

In [125]:
# As you can see above, certain years are missing from the columns
missing_years = [1997, 1999, 2001]

In [126]:
def insert_missing_years(df, missing_years):
    for year in missing_years:
        prev_year = str(year - 1)
        next_year = str(year + 1)

        # Check if previous and next year columns exist
        if prev_year in df.columns and next_year in df.columns:
            df[prev_year] = pd.to_numeric(df[prev_year], errors='coerce')
            df[next_year] = pd.to_numeric(df[next_year], errors='coerce')
            df[str(year)] = (df[prev_year] + df[next_year]) / 2
        else:
            print(f"Cannot compute for year {year}, adjacent years data are not available.")
        
    # Sort the columns
    non_year_cols = ["Country Name", "Country Code", "Series Name", "Series Code"]
    year_cols = [str(i) for i in sorted(int(col) for col in df.columns if col.isdigit())]
    cols = non_year_cols + year_cols
    
    return df.reindex(columns=cols)

pop_stab = insert_missing_years(pop_stab, missing_years)

In [127]:
pop_stab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 40 to 203
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  3 non-null      object 
 1   Country Code  3 non-null      object 
 2   Series Name   3 non-null      object 
 3   Series Code   3 non-null      object 
 4   1996          3 non-null      float64
 5   1997          3 non-null      float64
 6   1998          3 non-null      float64
 7   1999          3 non-null      float64
 8   2000          3 non-null      float64
 9   2001          3 non-null      float64
 10  2002          3 non-null      float64
 11  2003          3 non-null      float64
 12  2004          3 non-null      float64
 13  2005          3 non-null      float64
 14  2006          3 non-null      float64
 15  2007          3 non-null      float64
 16  2008          3 non-null      float64
 17  2009          3 non-null      float64
 18  2010          3 non-null      f

In [130]:
pop_stab.head()

,Country Name,Country Code,Series Name,Series Code,1996,1997,1998,1999,2000,2001,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
40,China,CHN,Political Stability and Absence of Violence/Te...,PV.EST,-0.097849,-0.245977,-0.394105,-0.302011,-0.209917,-0.268772,...,-0.538782,-0.543790,-0.522584,-0.550295,-0.499395,-0.231018,-0.300023,-0.256689,-0.329403,-0.481860
101,"Korea, Rep.",KOR,Political Stability and Absence of Violence/Te...,PV.EST,0.566771,0.540322,0.513873,0.455959,0.398045,0.319909,...,0.267103,0.278579,0.108569,0.161014,0.163469,0.385621,0.637073,0.551206,0.566344,0.662571
203,United States,USA,Political Stability and Absence of Violence/Te...,PV.EST,0.935006,0.907309,0.879613,0.981211,1.082809,0.684143,...,0.632442,0.643073,0.582420,0.678062,0.402303,0.291817,0.421418,0.134975,0.033438,0.004954


### HDI and Life Expectancy

In [174]:
def process_dataframe(df):
    # get the basic information
    country_code = df.loc[df['key'] == 'ISO3', 'value'].values[0]
    country_name = df.loc[df['key'] == 'HDR Country Name', 'value'].values[0]
    # filter rows for HDI and Life Expectancy
    hdi_rows = df[df['key'].str.startswith('Human Development Index (value)')]
    life_exp_rows = df[df['key'].str.match(r'^Life Expectancy at Birth \(years\) \(\d{4}\)$')]
    # create the two empty dataframes
    hdi_df = pd.DataFrame({'Country Name': country_name, 'Country Code': country_code}, index=[0])
    life_exp_df = pd.DataFrame({'Country Name': country_name, 'Country Code': country_code}, index=[0])
    # fill the dataframes
    for index, row in hdi_rows.iterrows():
        year = row['key'].split(' ')[-1]
        year = year.replace('(', '').replace(')', '')  # remove parentheses
        hdi_df[year] = row['value']

    for index, row in life_exp_rows.iterrows():
        year = row['key'].split(' ')[-1]
        year = year.replace('(', '').replace(')', '')  # remove parentheses
        life_exp_df[year] = row['value']
    
    return hdi_df, life_exp_df


In [177]:
# List of your dataframes
hdi_le_lists = [china_hdi_le, s_korea_hdi_le, us_hdi_le]

# Apply the function to each dataframe in the list and unzip the results
hdi_dfs, life_exp_dfs = zip(*[process_dataframe(df) for df in hdi_le_lists])

# Concatenate all the resulting dataframes
hdi_df = pd.concat(hdi_dfs)
life_exp_df = pd.concat(life_exp_dfs)


In [178]:
hdi_df

,Country Name,Country Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,China,CHN,0.484,0.492,0.504,0.515,0.525,0.535,0.545,0.554,...,0.709,0.717,0.725,0.733,0.74,0.747,0.755,0.762,0.764,0.768
0,Korea (Republic of),KOR,.737,.747,.754,.763,.773,.783,.793,.806,...,.897,.901,.906,.909,.912,.916,.919,.923,.922,.925
0,United States,USA,.872,.873,.878,.88,.884,.885,.887,.89,...,.916,.917,.919,.92,.922,.924,.927,.93,.92,.921


In [179]:
life_exp_df

,Country Name,Country Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,China,CHN,68.0052,68.1688,68.7345,69.2164,69.5201,70.008,70.2659,70.672,...,76.1923,76.4523,76.7173,76.9771,77.2177,77.2476,77.7444,77.968,78.0766,78.2107
0,Korea (Republic of),KOR,71.9401,72.4207,72.9564,73.283,73.61109999999999,73.9409,74.2757,74.97069999999999,...,81.3473,81.86369999999999,82.3781,82.5605,82.9259,83.2855,83.34269999999999,83.6557,83.60890000000001,83.6978
0,United States,USA,75.3699,75.5227,75.77760000000001,75.56699999999999,75.7377,75.8536,76.17529999999999,76.498,...,78.94410000000001,78.9507,79.0175,78.8694,78.84820000000001,78.82129999999999,78.9896,79.13800000000001,77.4144,77.1982
